In [119]:
%cd C:\Users\steff\Desktop\Gyro\vaults

C:\Users\steff\Desktop\Gyro\vaults


Got some weird differences in errors for a CEMM with l=1 where rotation still made a difference for errors.

In [195]:
from tests.cemm import cemm as mimpl
from tests.support.types import *
from tests.support.quantized_decimal import QuantizedDecimal as D

import brownie
brownie.reverts = None  # Hack so we can import util -.-
from tests.cemm.util import params2MathParams

from math import acos, pi, sqrt

In [121]:
def prt(x):
    print(f"{x:,}")

def fmt(x):
    return f"{x:,}"

In [122]:
balances=(327937, 501870798)
amountIn=D('1.000000000000000000')
tokenInIsToken0=False

In [123]:
params_nr=CEMMMathParams(alpha=D('3.044920011516668204'), beta=D('3.045020011516668204'), c=D('1'), s=D('0'),
                                  l=D('1.000000000000000000'))

In [124]:
params_r=CEMMMathParams(alpha=D('3.044920011516668204'), beta=D('3.045020011516668204'),
                                  c=D('0.302739565961017365'), s=D('0.953073320999877183'),
                                  l=D('1.000000000000000000'))

In [125]:
acos(params_r.c)/(2*pi)*360

72.37777765386596

In [126]:
cemm_nr = mimpl.CEMM.from_x_y(*balances, params2MathParams(params_nr))
cemm_r  = mimpl.CEMM.from_x_y(*balances, params2MathParams(params_r))

In [127]:
cemm_r.r, cemm_nr.r

(Decimal('54368166233512.039832941677650911'),
 Decimal('54368166235716.228646921494397316'))

In [128]:
prt(cemm_r.r)

54,368,166,233,512.039832941677650911


In [129]:
list(map(fmt, balances))

['327,937', '501,870,798']

In [130]:
fmt(cemm_r.a), fmt(cemm_r.b)

('51,654,031,855,794.511790877256851412',
 '16,963,948,646,553.821236141843501679')

In [131]:
cemm_r.a - cemm_nr.a

Decimal('-2094.149602071003245416')

In [132]:
cemm_r.b - cemm_nr.b

Decimal('-687.749802647502037873')

In [133]:
cemm_r.trade_y(amountIn, mock=True)

Decimal('-0.331721979533495728')

In [134]:
cemm_nr.trade_y(amountIn, mock=True)

Decimal('-0.328374841550181622')

Internal consistency:

In [135]:
def delta_invariant_from_delta_balances(cemm: mimpl.CEMM, dx, dy):
    """This *should* be >= 0 for any trade, and = 0 when there are no fees"""
    cemm1 = mimpl.CEMM.from_x_y(cemm.x + dx, cemm.y + dy , cemm.params)
    return cemm1.r - cemm.r

In [136]:
def delta_value_from_delta_balances(cemm: mimpl.CEMM, dx, dy):
    """This *should* be >= 0 for any trade, and = 0 when there are no fees"""
    cemm1 = mimpl.CEMM.from_x_y(cemm.x + dx, cemm.y + dy , cemm.params)
    reldr = (cemm1.r - cemm.r) / cemm.r
    return cemm.params.beta * reldr * cemm.x + reldr * cemm.y

In [137]:
delta_value_from_delta_balances(cemm_nr, cemm_nr.trade_y(amountIn, mock=True), amountIn)

Decimal('0.000092101028333524')

In [138]:
delta_value_from_delta_balances(cemm_r, cemm_r.trade_y(amountIn, mock=True), amountIn)

Decimal('-0.010099999599411897')

In [139]:
delta_value_from_delta_balances(cemm_nr, amountIn, cemm_nr.trade_x(amountIn, mock=True))

Decimal('0.000092101028333524')

In [140]:
delta_value_from_delta_balances(cemm_r, amountIn, cemm_r.trade_x(amountIn, mock=True))

Decimal('-0.010099999599411897')

In [93]:
delta_invariant_from_delta_balances(cemm_r, amountIn, cemm_r.trade_x(amountIn, mock=True))

Decimal('-1091.970424900237683119')

## Digging into inaccuracy

In [161]:
mycemm = cemm_nr

In [207]:
r = mycemm.r
chi = (mycemm.a / r, mycemm.b / r)
params = mycemm.params
mybalances = (mycemm.x, mycemm.y)
atvec = params.A_times(*mybalances)

In [208]:
achi = params.A_times(*chi)
achiachi = mimpl.scalarprod(*achi, *achi)
atachi = mimpl.scalarprod(*atvec, *params.A_times(*chi))
atat = mimpl.scalarprod(*atvec, *params.A_times(*mybalances))

In [209]:
achi

(Decimal('0.950078610963991580'), Decimal('0.312019878943376936'))

In [210]:
tuple(map(fmt, (achiachi, atachi, atat)))

('1.000005771867107258',
 '156,905,231.566619678191885388',
 '251,874,405,427,832,773.000000000000000000')

In [205]:
def to_polar(x, y):
    l = (x**2 + y**2).sqrt()
    angle_degrees = acos(x/l)/(2*pi)*360
    return l, angle_degrees

In [211]:
to_polar(*achi)

(Decimal('1.000002885929389334'), 18.18094571179936)

In [213]:
to_polar(*atvec)

(Decimal('501870905.141783222101605509'), 89.962561273844)

In [190]:
prt(1/(achiachi - 1))

173,254.162200394615937975


In [176]:
prt(atachi**2)
prt((achiachi - 1) * atat)

24,619,251,692,974,544.312744737973605013
1,453,785,595,849.073841375678566434


In [183]:
d = atachi**2 - (achiachi - 1) * atat
prt(d)

24,617,797,907,378,695.238903362295038579


In [188]:
prt(d.sqrt())

156,900,598.811408923312449457


In [191]:
params.beta - params.alpha

Decimal('0.000100000000000000')

In [192]:
D('3.045020011516668204') - D('3.044920011516668204')

Decimal('0.000100000000000000')

In [193]:
D('5.190714635386387361') - D('5.189714635386387361')

Decimal('0.001000000000000000')

# Mirrored

In [61]:
balances1=(balances[1], balances[0])
beta1 = D(1)/params_nr.alpha
alpha1 = D(1)/params_nr.beta

In [62]:
def delta_value_from_delta_balances(cemm: mimpl.CEMM, dx, dy):
    """This *should* be >= 0 for any trade, and = 0 when there are no fees"""
    cemm1 = mimpl.CEMM.from_x_y(cemm.x + dx, cemm.y + dy , cemm.params)
    reldr = (cemm1.r - cemm.r) / cemm.r
    return reldr * cemm.x + reldr * cemm.y / cemm.params.alpha

In [63]:
params_nr=CEMMMathParams(alpha=alpha1, beta=beta1, c=D('1'), s=D('0'),
                                  l=D('1.000000000000000000'))

In [64]:
params_r=CEMMMathParams(alpha=alpha1, beta=beta1,
                                  c=D('0.302739565961017365'), s=D('0.953073320999877183'),
                                  l=D('1.000000000000000000'))

In [65]:
cemm_nr = mimpl.CEMM.from_x_y(*balances1, params2MathParams(params_nr))
cemm_r  = mimpl.CEMM.from_x_y(*balances1, params2MathParams(params_r))

In [66]:
cemm_r.trade_y(amountIn, mock=True)

Decimal('-0.338552096568681593')

In [67]:
cemm_nr.trade_y(amountIn, mock=True)

Decimal('-0.328291215436218165')

In [68]:
delta_value_from_delta_balances(cemm_nr, cemm_nr.trade_y(amountIn, mock=True), amountIn)

Decimal('0.000124634233282244')

In [69]:
delta_value_from_delta_balances(cemm_r, cemm_r.trade_y(amountIn, mock=True), amountIn)

Decimal('-0.010136246495037787')

In [70]:
delta_value_from_delta_balances(cemm_nr, amountIn, cemm_nr.trade_x(amountIn, mock=True))

Decimal('0.000124634233282244')

In [71]:
delta_value_from_delta_balances(cemm_r, amountIn, cemm_r.trade_x(amountIn, mock=True))

Decimal('-0.010136246495037787')